In [606]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import sys
import time

In [690]:
SPOTIPY_CLIENT_ID='bee50ef8746e41e293d2b3eaabc012dd'
SPOTIPY_CLIENT_SECRET='a13911965b1246959864149c52a26749'
SPOTIPY_REDIRECT_URI = "https://jperezllo.con/callback/"

token = util.prompt_for_user_token(username,scope,client_id=SPOTIPY_CLIENT_ID
                           ,client_secret=SPOTIPY_CLIENT_SECRET
                           ,redirect_uri=SPOTIPY_REDIRECT_URI)

In [11]:
auth_manager = SpotifyClientCredentials(client_id = SPOTIPY_CLIENT_ID, client_secret = SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [696]:
scope = 'user-top-read'

sp = spotipy.Spotify(auth=token)
results = sp.current_user_top_tracks()
tracks = results['items']

while results['next']:
    results = sp.next(results)
    tracks.extend(results['items'])

my_top_songs = [tracks[item]['name'] for item in range(0, len(tracks))]
    

In [714]:
scope = 'playlist-read-private'

token = util.prompt_for_user_token(username,scope,client_id=SPOTIPY_CLIENT_ID
                           ,client_secret=SPOTIPY_CLIENT_SECRET
                           ,redirect_uri=SPOTIPY_REDIRECT_URI)

sp = spotipy.Spotify(auth=token)
results = sp.current_user_playlists(limit=50)
playlists = results['items']
while results['next']:
    results = sp.next(results)
    playlists.extend(results['items'])

my_playlists =[playlists[item]['name'] for item in range(0, len(playlists))]


### Getting all artists id on my JP playlist

In [12]:
jp_id = '3FInOSJaKv5QzLQrd3lCgA'

In [69]:
chill_out = '00X3SYeYrgAdtgrKt8QzsJ'

In [266]:
my_playlists = sp.user_playlists('perezllo', limit = 10)

https://stackoverflow.com/questions/39086287/spotipy-how-to-read-more-than-100-tracks-from-a-playlist

In [712]:
#def get_playlists(username):
   # my_playlists = sp.user_playlists('perezllo')['items']
   # playlist_id = []
   # for number in range(0, len(my_playlists)):
        #playlist_id[my_playlists[number]['name']] = my_playlists[number]['id']
        #playlist_id.append(my_playlists[number]['id'])
    #return playlist_id


def get_playlists():
    token = util.prompt_for_user_token(username,scope,client_id=SPOTIPY_CLIENT_ID
                               ,client_secret=SPOTIPY_CLIENT_SECRET
                               ,redirect_uri=SPOTIPY_REDIRECT_URI)

    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_playlists(limit=50)
    playlists = results['items']
    while results['next']:
        results = sp.next(results)
        playlists.extend(results['items'])

    my_playlists =[playlists[item]['id'] for item in range(0, len(playlists))]
    return my_playlists

In [512]:
def playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    track_info = {}
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    for item in range(0, len(tracks)):
        track_info[tracks[item]['track']['name']]={"id":tracks[item]['track']['id']}
    return track_info

In [573]:
def get_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    track_info = {}
    errors = []
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
        
    for item in range(0, len(tracks)):
        
        track_id = tracks[item]['track']['id']
        keys_to_remove =["type", "uri", "track_href","analysis_url", "time_signature","id"]
        audio_features =  sp.audio_features(track_id)
        audio_features = audio_features[0]
        for key in keys_to_remove:
            try: 
                audio_features.pop(key, None)
            except:
                pass
        track_info[tracks[item]['track']['name']] = {"id":track_id}
        try:
            track_info[tracks[item]['track']['name']].update(audio_features)
        except:
            pass
    return  track_info


In [562]:
restuls = sp.playlist_tracks('5U2MYFAcQ3yT7XbHYgGPmb')

In [574]:
s = get_playlist_tracks('5U2MYFAcQ3yT7XbHYgGPmb')

In [588]:
def get_all_track_info(username):
    playlists = get_playlists(username)
    full_tracks = {}
    for playlist_id in playlists: 
        full_tracks.update(get_playlist_tracks(playlist_id))
    return pd.DataFrame.from_dict(full_tracks, orient='index')
        
        
        
        

In [589]:
result = get_all_track_info('perezllo')

### Returning names of related artiststs

In [ ]:
def get_related_artists(artist_id):
    results = sp.artist_related_artists(artist_id)
    artists = results['artists']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
        artists_id = [item['track']['artists'][0]['id'] for item in tracks]
    return artists_id

In [64]:
related_artists = sp.artist_related_artists('2hazSY4Ef3aB9ATXW7F5w3')

In [206]:
def related_artists_id(artists_id_list):
    # for artist_id in artist_id_list:
    related_artists = []
    related_artists_id = []
    
    for artist_id in artists_id_list: 
        related_artists.append(sp.artist_related_artists(artist_id))
    
    for n_1 in range(0, len(related_artists)):
        for n_2 in range(0, len(related_artists[n_1]['artists'])):
            if related_artists[n_1]['artists'][n_2]['name'] not in related_artists_id: 
                related_artists_id.append(related_artists[n_1]['artists'][n_2]['id'])
            else: 
                pass
            
            
    return related_artists_id
        
  #  return len(related_artists) # related_artists_id
                

In [207]:
related_artists_id = related_artists_id(artist_id)